In [ ]:
pip install mysql-connector


In [ ]:
import mysql.connector

connection = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="password",
    database="REDBUS",
    auth_plugin="mysql_native_password"  # Explicitly specify the plugin
)


In [ ]:
import pandas as pd
import mysql.connector

# Establish the connection to MySQL with the correct authentication plugin
try:
    mydb = mysql.connector.connect(
        host="localhost",           # Specify the host, commonly localhost
        user="root",                # Your MySQL username
        password="password",      # Your MySQL password
        auth_plugin="mysql_native_password"  # Specify the authentication plugin
    )
    print("Connected to MySQL successfully!")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit()

# Create a cursor object to execute SQL queries
cursor = mydb.cursor()

# Create the database 'REDBUS' if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS REDBUS")
print("Database 'REDBUS' created (or already exists)")

# Use the 'REDBUS' database
cursor.execute("USE REDBUS")

# Define the schema for the MySQL table with the 'state' column
schema = """
CREATE TABLE IF NOT EXISTS bus_routes (
    id INT PRIMARY KEY AUTO_INCREMENT,
    route_name TEXT,
    route_link TEXT,
    busname TEXT,
    bustype TEXT,
    departing_time TIME,
    duration TEXT,
    reaching_time TIME,
    price DECIMAL(10,2),
    seats_available INT,
    star_rating FLOAT,
    state TEXT  # Adding the 'state' column
);
"""
cursor.execute(schema)
print("Table 'bus_routes' created successfully!")

# List of CSV files to read
csv_files = [
    'himachal_bus_details.csv', 'south_bengal_bus_details.csv', 'uttar_pradesh_bus_details.csv',
    'andhra_pradesh_bus_details.csv', 'WEST_BENGAL_bus_details.csv', 'Telangana_bus_details.csv',
    'rajasthan_bus_details.csv', 'kerala_bus_details.csv', 'chandigarh_bus_details.csv', 'punjab_bus_details.csv'
]

# State mapping for each CSV file
state_mapping = {
    'himachal_bus_details.csv': 'Himachal Pradesh',
    'south_bengal_bus_details.csv': 'West Bengal',
    'uttar_pradesh_bus_details.csv': 'Uttar Pradesh',
    'andhra_pradesh_bus_details.csv': 'Andhra Pradesh',
    'WEST_BENGAL_bus_details.csv': 'West Bengal',
    'Telangana_bus_details.csv': 'Telangana',
    'rajasthan_bus_details.csv': 'Rajasthan',
    'kerala_bus_details.csv': 'Kerala',
    'chandigarh_bus_details.csv': 'Chandigarh',
    'punjab_bus_details.csv': 'Punjab'
}

# Reading and combining data from all CSV files
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    
    # Add the 'state' column based on the file name
    df['state'] = state_mapping[file]
    
    # Renaming columns to match the table schema
    df.rename(columns={
        "Route_Name": "route_name",
        "Route_Link": "route_link",
        "Bus_Name": "busname",
        "Bus_Type": "bustype",
        "Departing_Time": "departing_time",
        "Duration": "duration",
        "Reaching_Time": "reaching_time",
        "Star_Rating": "star_rating",
        "Price": "price",
        "Seat_Availability": "seats_available"
    }, inplace=True)

    # Convert time columns to appropriate format
    df['departing_time'] = pd.to_datetime(df['departing_time'], format='%H:%M', errors='coerce').dt.time
    df['reaching_time'] = pd.to_datetime(df['reaching_time'], format='%H:%M', errors='coerce').dt.time
    
    # Clean 'price' column to numeric values
    df['price'] = df['price'].replace(r'INR ', '', regex=True).replace(r',', '', regex=True).astype(float)
    
    # Extract 'seats_available' from Seat_Availability
    df['seats_available'] = df['seats_available'].str.extract(r'(\d+)').astype(int)
    
    # Append the dataframe to the list
    dfs.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Drop rows with missing critical data
critical_columns = ['route_name', 'route_link', 'busname', 'bustype', 'departing_time', 'reaching_time', 'price', 'seats_available', 'star_rating']
combined_df = combined_df.dropna(subset=critical_columns)

# Prepare data for insertion into MySQL
columns_to_insert = [
    "route_name", "route_link", "busname", "bustype",
    "departing_time", "duration", "reaching_time",
    "price", "seats_available", "star_rating", "state"
]

# Insert data into the 'bus_routes' table
insert_query = f"""
    INSERT INTO bus_routes ({', '.join(columns_to_insert)}) 
    VALUES ({', '.join(['%s'] * len(columns_to_insert))})
"""
data = combined_df[columns_to_insert].values.tolist()

# Execute the insert query
try:
    cursor.executemany(insert_query, data)
    mydb.commit()
    print(f"Data inserted into table 'bus_routes' successfully!")
except mysql.connector.Error as err:
    print(f"Error inserting data: {err}")

# Close the database connection
cursor.close()
mydb.close()
print("MySQL connection closed.")
